# Raspberry Pi Data Logger Interface via Jupyter/Python<a class="tocSkip">

&copy; 2020 by Michael Stanley (Mike.Stanley@ieee.org)<BR>
Last revised: 29 August 2020

# Introduction
This notebook uses a number of Python libraries to implement a simple interface for capturing sensor information
from a Raspberry Pi.    Graphics are provided using the bqplot library, and GUI widgets are created
using the standard ipywidgets library.  Communication to the Pi is via a simple streaming socket interface.
    
This notebook is intended both as proof of concept and as a teaching aide.  It is NOT intended for production use in collecting sensor data.  In that case, a non-Jupyter implemention makes more sense.  Please use the notebook in the spirit in which it is intended.
    
This draft of the Notebook is intended to work in conjunction with a standard Raspberry Pi (RPi 3 Model B used for development) coupled with an RPi Sense Hat.
![Raspberry Pi Sense Hat](../Images/SenseHat.png)  
![Raspberry Pi Model 3B](../Images/RPi.png)
# External code Imports
## Import from standard and 3rd party libraries

In [1]:
import ipywidgets as widgets        # Standard widget library for GUI construction 
from ipywidgets import interact, interactive, fixed, interact_manual
from pythreejs import pi, Preview   # Graphics library (used for 3D display)
import binascii                     # convert binary data to printable form
import threading                    # support for multithreading
import numpy as np                  # mathematical library
import time                         # used for the sleep function
import math                         # standard math functions
#import yappi                       # you can optionally use yappi to profile parts of the code
import bqplot as bq                 # bqplot provides an object oriented plotting function with many capabilities
import collections                  # used to create dictionaries
from traitlets import link          # used for linking plot selection interaction functions
from IPython.display import clear_output, display  # used to implement GUI functions
from ipyfilechooser import FileChooser             # 3rd party code used to implement a file selection function
import os                           # primarily used to get the current working directory on the PC
import socket                       # Used for communications with the Raspberry Pi
import json                         # Data is sent via JSON packets, which are easily decoded here

## Import other modules of this application
Some libraries are built as reusable modules that can be imported into multiple Jupyter Notebooks

In [2]:
from OutputWidgetHandler import *           # ipywidgets adaptation for use with standard python logging functions

# Central Status Logging Facility

Because we have multiple GUI objects as well as multiple threads, a standard mechansim is required to send 
messages to the user. This is provided using the logger object.  Reference OutputWidgetHandler.py in this same 
directory for details of the implementation.

Example logger calls include:
*    logger.debug("Test Debug Message")
*    logger.info("Test Info Message")
*    logger.warning("Test Warning Message")
*    logger.error("Test Error Message")
*    logger.critical("Test Critical Message")

Logger messaged are automatically sent to the GUI console window.  That window can be cleared by clicking the "Clear Console" button on the "Main" tab of the GUI.

## Setup the logger

In [3]:
logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s - %(threadName)-10s %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)
#logger.setLevel(4)

# Custom class for tracking data rates
The TimeKeeper class was designed to keep track of the number of packets sent over time and the instantaneous and average data rates over time.  The current Raspberry Pi embedded code does NOT use a hard real-time clock to ensure data rates.  We send data as quickly as possible.  So this class can be used to compute effective rates.

In [4]:
import time
import threading
class TimeKeeper():
    def __init__(self):
        self.lock = threading.Lock()
        self.reset()
    def reset(self):
        with self.lock:
            self.current_rate=0
            self.average_rate=0
            self.packet_counter=0
            self.last_packet_counter=0
            self.last_time=time.time()
            self.start_time=self.last_time
    def incrementPacket(self):
        with self.lock:
            self.packet_counter += 1
    def snapshot(self):
        with self.lock:
            newTime = time.time()
            deltaTime = newTime - self.last_time
            self.last_time = newTime
            deltaPackets = self.packet_counter - self.last_packet_counter
            self.last_packet_counter = self.packet_counter
            if (deltaTime>0):
                self.current_rate = deltaPackets/deltaTime
            deltaTime = newTime - self.start_time
            if (deltaTime>0):
                self.average_rate = self.packet_counter/deltaTime
            return (self.packet_counter, self.current_rate, self.average_rate)

# Update GUI Function Called During Serial Streaming
Data streams to/from the Raspberry Pi and updates to the GUI are done in two separate background tasks.  In this way, GUI update rates can be separately throttled so that GUI updates (which are slow) do not starve the serial stream of CPU cycles.  This is a fairly standard way of managing tradeoffs in these types of application.
We keep a list of functions called "gui_updaters" which are periodically called to update various portions of the GUI.  This allows us to keep individual GUI update functions simple, but still executed in a controlled fashion.

In [5]:
gui_updaters = []     # List of functions to be called periodically to update the GUI
                      # This list will be added to in the code that follows  
stop_gui_updates = False  # Set to True to stop the GUI updates (normally done when the serial port is inactive)
skip_plot_updates = False  # Set to True to skip plot updates BUT NOT DISABLE serial port or GUI update thread
gui_update_interval = 0.5  # Experimentally, a half second update interval seems to work best
def update_gui(socketClient):
    global logger, gui_update_interval, stop_gui_updates, gui_updaters
    try:
        while not stop_gui_updates:
            with socketClient.lock:
                for item in gui_updaters:
                    item()
            time.sleep(gui_update_interval)
        logger.info("Exiting update_gui() loop")
    except Exception as ex:
        logger.error("Exception in function update_gui(): ", str(ex))

# Socket Client
Python has a very nice (and simple) socket library available via the "socket" module.  We have used that to create a simple streaming interface for communicating with the Raspberry Pi.  We assign a fixed IP address to the Pi using the fixed IP address feature of a wireless router.  This is a standard feature available on modern routers.
The communications process is:
* Upon startup, the Raspberry Pi connects to the wireless network and is assigned a (fixed) IP address
* The Pi will output this address to the LED panel on the Sensor hat
* That same address should be assigned to the ipAddress text field defined later in this notebook.
* Clicking the "Connect to Raspberry Pi" button will initiate a socket
* The Pi sends a connection confirmation
* The Pi sends a continuous stream of sensor packets (in JSON format) until the connection is terminated either by this Notebook or the Pi itself.
* Packets are decoded using the "read_packet" function, and GUI updates done accordingly.

The only "public" interfaces for this client are the constructor, connect() and stop().

In [6]:
class SocketClient():
    global logger
    def __init__(self, headersize=10, typesize=3):
        self.headersize=headersize
        self.typesize=typesize
        self.port = 5556              # Must match port number used on code running on the Raspberry PI
        self.lock = threading.Lock()  # used for coordination between threads
        self.connection_ok = False

    def open(self, board_address = '192.168.0.253'):
        try:
            self.host = board_address
            self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
            self.s.connect((self.host, self.port)) 
            self.connection_ok = True        
            self.thread = threading.Thread(name="Reading", target=self.connect)
            self.thread.setDaemon(True)
            self.thread.start() # Kick off the separate thread running the connect function below
        except Exception as ex:
            self.connection_ok=False
        return(self.connection_ok)
    
    def process_packet(self, msg, new_msg, full_msg):
        if new_msg:
            self.packet_type = int(msg[:self.typesize])
            self.msgLen = int(msg[self.typesize:self.headersize+self.typesize])
            new_msg=False
        full_msg += msg.decode("utf-8")
        payload_length = len(full_msg)-self.headersize-self.typesize
        if payload_length >= self.msgLen:
            payloadStr = full_msg[self.headersize+self.typesize:self.headersize+self.typesize+self.msgLen]
            if (self.packet_type==2):
                self.numPackets+=1
                #print('Packet type 2 received: {0} '.format(payloadStr))
                packet = json.loads(payloadStr)
                #print('Packet {0} received'.format(numPackets))
                #print('Packet type 2 received: {0} '.format(str(packet)))
                read_packet(packet)
            else:
                logger.info("Message received: {0}".format(payloadStr))
        if payload_length == self.msgLen:
            new_msg=True
            full_msg=''
        elif payload_length > msgLen:
            logger.Info("Message length exceeded expected: {0}. Processing further...".format(full_msg[self.headersize:]))
            msg = full_msg[self.headersize+self.typesize+self.msgLen:]
            new_msg=True
            full_msg=''
            new_msg, full_msg = self.process_packet(msg, new_msg, full_msg)
        return(new_msg, full_msg)

    def connect(self):
        full_msg=''
        new_msg = True
        self.numPackets=0
        while self.connection_ok==True:
            try:
                msg = self.s.recv(1024)
                new_msg, full_msg = self.process_packet(msg, new_msg, full_msg)
            except Exception as ex:
                connection_ok = False
            time.sleep(0.001)
        self.s.close()
        logger.info('Connection closed')
    def stop(self):
        self.connection_ok=False
        
socketClient = SocketClient()

# GUI Component Definitions
Technically speaking, you really don't need a graphical user interface to do data collection.  But it helps tremendously to ensure that connections are properly made and that data received looks good in real-time, rather than waiting until a lengthly data collection session has been completed.

The GUI interface includes a "Main" tab that can be used to specify non-default IP addresses for the Raspberry PI, fields to display recently received sensor values.  Also included are tabs for displaying charts for each sensor type.  The charts are "smart" and include the ability to connect/disconnect, turn data logging on/off, zoom in both X and Y dimensions and do separate dumps of subsegments of the input waveforms.

## Communications & Plot Control Functions
The serial port operates in a separate thread managed by the SocketClient class defined above.  That thread is started/stopped via the toggle_serial_port() function callback to the toggle1 button that follows below.  Notice how the toggle1 button description is changed depending upon the mode of operation.

In [7]:
def stop_data():
    global stop_gui_updates, toggle1, socketClient, logger
    # Disable serial port
    with socketClient.lock: # lock prevents a race condition that can occur due to async stop & read functions
        stop_gui_updates = True;
        logger.debug("STOP REQUEST ACKNOWLEDGED")
        socketClient.stop()
        toggle1.description="Connect to Raspberry Pi"
        logger.debug("Board connection has been closed")

def toggle_serial_port(change):
    # Callback for serial port enable/disable button
    global logger, socketClient, ipAddress, stop_gui_updates
    newState = change['new']
    if newState:
        # Enable serial port
        logger.debug("Button callback: Opening connection to board")
        sts = socketClient.open(board_address = ipAddress.value)
        if sts:
            logger.debug("Socket to board successfully opened")
            stop_gui_updates = False;
            toggle1.description="Close connection"
            logger.debug("Starting second thread for GUI updates")
            t2 = threading.Thread(name="GuiUpdate", target=update_gui, kwargs=dict(socketClient=socketClient))
            t2.setDaemon(True)
            t2.start()
            logger.debug("GUI update thread started.")            
        else:
            logger.error("ERROR!  Could not open connection to board")
    else:
        stop_data()
    return()

## Port Enable/Disable
Here is the actual toggle1 button used to enable/disable the connection to the serial board.  Note how the callback above is tied to the button using it's observe method.

In [8]:
toggle1=widgets.ToggleButton(
    value=False,
    description='Connect to Raspberry Pi',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click this button to start/stop receiving data from your Raspberry Pi',
    icon='',
    layout=widgets.Layout(width='200px', height='auto')
) # note we are not displaying the widget yet

toggle1.observe(toggle_serial_port, 'value') # This ties in the callback function

## 7.3 Field used to specify IP address for the Raspberry Pi
Note that the value field should be changed to default address of your Pi board.  But if you forget, you can overwrite the value in the GUI prior to opening the socket location.

In [9]:
ipAddress = widgets.Text(
    value='192.168.0.253',  # This should match the IP address assigned to your Raspberry Pi
    description='IP Address:',
    disabled=False   
)
#display(ipAddress)

Text(value='192.168.0.253', description='IP Address:')

## Plot freeze/unfreeze control
The plot freeze control can suspend plot updates without effecting data streaming operations.

In [10]:
def toggle_plot_freeze(change):
    # Callback for serial port enable/disable button
    global skip_plot_updates
    newState = change['new']
    if newState:
        skip_plot_updates = False
        toggle2.description="Freeze Plots"
    else:
        skip_plot_updates = True;
        toggle2.description="Re-enable Plots"
    return()

toggle2=widgets.ToggleButton(
    value=True,
    description='Freeze Plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click this button to freeze/start plots without affecting sensor communications',
    icon=''
) # note we are not displaying the widget yet

toggle2.observe(toggle_plot_freeze, 'value') # This ties in the callback f

## Data Logger
The act of recording sensor data to .csv files is encapsulated within the LoggingController class.  Notice that this class incorporates both GUI and functional components of the data logger.  It looks like an ipywidgets HBOX that contains all the functionality needed for specifying what output file to write and when to write it.

"Public" methods are the constructor, log() and loggerStartStop() functions.  One of the GUI sub-components here is a file selection widget derived from the 3rd party ipyfilechooser Python library.

Data logs are in .csv format and include the following columns of data:
* sample number
* sample time
* X-axis acceleration in g's
* Y-axis acceleration in g's
* Z-axis acceleration in g's
* X-axis magnitometer reading in microTeslas
* Y-axis magnitometer reading in microTeslas
* Z-axis magnitometer reading in microTeslas
* X-axis gyro reading in radians per second
* Y-axis gyro reading in radians per second
* Z-axis gyro reading in radians per second
* Temperature in Celcius
* Relative Humidity (rH) in %
* Pressure in inches of Mercury (the Raspberry PI Sense Hat returns pressure in Millibars, but the server on the Pi converts it to inches of Mercury before transmission.

See https://pythonhosted.org/sense-hat/api/ for additional details of the Sense Hat API.

In [11]:
logging_enabled = False  # Set to True to enable logging

class LoggingController(widgets.HBox):
    def __init__(self):
        self.lf = None
        self.logging_enabled = False;
        self.logIndex=0
        self.toggle=widgets.ToggleButton(
            value=False,
            description='Start Logging',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click this button to turn sensor logging on and off',
            icon=''
        ) # note we are not displaying the widget yet
        
        # Create new FileChooser:
        # Path: current directory
        # File: log.csv
        # Title: <b>Logger Output File</b>
        # Show hidden files: no
        # Use the default path and filename as selection: yes
        self.file_dialog = FileChooser(
            os.getcwd(),
            filename='log.csv',
            title='<b>Logger Output File</b>',
            show_hidden=False,
            select_default=True
        )
        
        self.toggle.observe(self.loggerStartStop, 'value') # This ties in the callback f
        
        super().__init__(
            children = [self.toggle, self.file_dialog]
        )

    def log(self, packetTime, accelX, accelY, accelZ, magX, magY, magZ, gyroX, gyroY, gyroZ, temp, pressure, rH):
        if not self.lf==None:
            print('{0},  {1:10.4f}, {2:6.3f}, {3:6.3f}, {4:6.3f}, {5:6.3f}, {6:6.3f}, {7:6.3f}, {8:6.3f}, {9:6.3f}, {10:6.3f}, {11:6.3f}, {12:6.3f}, {13:6.3f}'\
                .format(self.logIndex, packetTime, accelX, accelY, accelZ, magX, magY, magZ, gyroX, gyroY, gyroZ, temp, rH, pressure), file=self.lf)
            self.logIndex += 1
   
    def loggerStartStop(self, change):
        # Callback for logging start/stop button
        newState = change['new']
        if newState:
            try:
                self.lf = open(self.file_dialog.selected, 'wt')
                print('Index, Time, accelX, accelY, accelZ, magX, magY, magZ, gyroX, gyroY, gyroZ, temp, rH, pressure', file=self.lf)
                self.logging_enabled = True
                self.toggle.description="Stop Logging"
            except Exception as ex:
                logger.error("Could not open logger output file.  Check to see if it is open elsewhere. ", str(ex))
        else:
            if not self.lf==None:
                self.lf.close()
                self.lf = None
                self.logIndex = 0
            self.logging_enabled=False
            self.toggle.description="Start Logging"
        return()

loggingController = LoggingController()

## Button and Callback for Clearing Console
Clearing past messages from the console is trivially implemented with this button and callback.

In [12]:
def on_button1_clicked(b):
    handler.clear()  

button1 = widgets.Button(
    description='Clear Console',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon=''
)
button1.on_click(on_button1_clicked)

## Accel/Mag/Gyro/Quaternion Values Table
The "Main" tab of the GUI contains a table that displays recently received sensor values for the accelerometer, magnetometer and gyroscope.  That table is defined using an ipywidgets GridspecLayout function.

### Table Definition

In [13]:
from ipywidgets import GridspecLayout
colNames = ['X', 'Y', 'Z', 'VM']
rowNames = ["Accel (g's)", "Mag (uT's)", "Gyro (dps)"]
def table1(colNames, rowNames):
    grid = GridspecLayout(4, 5, grid_gap='0px', width='75%')
    rowNums = range(1,len(rowNames)+1)
    colNums = range(1,len(colNames)+1)
    for i in rowNums:
        for j in colNums:
            grid[i, j] = widgets.FloatText(value=0.0, layout=widgets.Layout(width='110px', height='auto'))
    for idx, colName in zip(colNums, colNames):
        grid[0, idx] = widgets.Label(value=colName, layout=widgets.Layout(width='110px', height='auto'))
    for idx, rowName in zip(rowNums, rowNames):
        grid[idx, 0] = widgets.Label(value=rowName, layout=widgets.Layout(width='110px', height='auto'))
    grid[0,0] = widgets.Text(value="0 / 0.0", layout=widgets.Layout(width='110px', height='auto'))
    return(grid)
sensor_grid1=table1(colNames, rowNames)
#display(sensor_grid1)  # Uncomment for debug

### Table Update Function

In [14]:
def update_sensor_grid_triaxial_row(row, x,y,z):
    global sensor_grid1
    sensor_grid1[row,1].value=x
    sensor_grid1[row,2].value=y
    sensor_grid1[row,3].value=z
    sensor_grid1[row,4].value=math.sqrt(x*x+y*y+z*z)
def update_accel_row(x,y,z):
    update_sensor_grid_triaxial_row(1,x,y,z)
def update_mag_row(x,y,z):
    update_sensor_grid_triaxial_row(2,x,y,z)
def update_gyro_row(x,y,z):
    update_sensor_grid_triaxial_row(3,x,y,z)
    
update_counter=TimeKeeper()

def update_counter_cell():
    global update_counter
    (packet_count, current_rate, average_rate) = update_counter.snapshot()
    s = '%d / %6.3f' % (packet_count, current_rate)
    sensor_grid1[0,0].value=s
def increment_counter():
    global update_counter
    logger.debug("updating counter")
    update_counter.incrementPacket()
gui_updaters.append(update_counter_cell)

## Temperature / Relative Humidity / Pressure Table
In addition to the tri-axis sensor table above, the "Main" tab of the GUI also contains a table that displays recently received sensor values for the temperature, humidity and relative humidity.  That table is defined using an ipywidgets GridspecLayout function (similar to the one above).  

The first column of the table below is empty, and is just used to visually space the two tables.
### Table Definition

In [15]:
rowNames = ["Temp (C)", "rH (%)", "Pressure (in Hg)"]
def table2(rowNames):
    grid = GridspecLayout(4, 3, grid_gap='0px', width='30%')
    rowNums = range(1,len(rowNames)+1)
    colNums = [2]
    for i in rowNums:
        for j in colNums:
            grid[i, j] = widgets.FloatText(value=0.0, layout=widgets.Layout(width='110px', height='auto'))
    for idx, rowName in zip(rowNums, rowNames):
        grid[idx, 1] = widgets.Label(value=rowName, layout=widgets.Layout(width='110px', height='auto'))
    return(grid)
sensor_grid2=table2(rowNames)
#display(sensor_grid2)  # Uncomment for debug

### Table Update Function

In [16]:
def update_sensor_trhp(temp, rH, pressure):
    global sensor_grid2
    sensor_grid2[1,2].value=temp
    sensor_grid2[2,2].value=rH
    sensor_grid2[3,2].value=pressure

## Assemble the contents of what will become the main tab
The graphical user interface utilizes a tabbed interface.  Individual tabs contents are defined and then assembled below.  We have a central logging console immediately below the tabbed interface, so that it is visible regardless of which tab is selected.  This console is written to using the standard Python logging library.

In [17]:
q_layout = widgets.Layout(width='110px')
style = {'description_width': 'initial'}
box_layout = widgets.Layout(align_items='center')
main_right_row1 = widgets.HBox([ipAddress, toggle1, button1])
main_right_row2 = widgets.HBox([sensor_grid1, sensor_grid2])
main_right_panel = widgets.VBox([main_right_row1, main_right_row2], layout=box_layout)
main = widgets.HBox([main_right_panel])
#display(main)

## Plot Implementations
### Circular Buffer
New data points are written directly from the serial port to very efficient circular buffers.  These are designed so that the buffers do not need to be copied for every new sample coming in.  Only when the plot is updated are those more time intensive tasks done.  A circular buffer class is defined to facilitate this.  

The implication of the above is that the length of the plots is fixed in time to a specific window into the past.  This keeps the plotting functions operating efficiently.  If you want the full history of data, you should use the data logger to write data to the file system as it arrives, and then post-process the resulting .csv data after the logging session is complete.

In [18]:
class plot_buffer():
    def __init__(self, size):
        self.b = np.zeros(size, dtype=np.float32)
        self.next = 0
        self.size=size
        self.full = False;
    def push(self, value):
        self.b[self.next]=value
        self.next += 1
        if self.next>=self.size:
            self.full = True
            self.next = 0
    def unwrapped(self):
        if (not self.full):
            return(self.b)
        else:
            return(np.concatenate((self.b[self.next:], self.b[0:self.next-1])))    

There are two very similar plot implementations.  The first version plots a single variable, the second version plots three values (X, Y & Z) plus the vector magnitude.  The first is essentially a stripped down version of the second, they have the same basic interfaces.

Both include a number of GUI controls plus the actual plots.  We use the bqplot library because it offers some very nice interactive plot features built right into the plot library itself.  These include pan/zoom and brush selection, plus the ability to update the plot points in time.

Both plot implementations plot values versus SAMPLE INDEX into the past.  This is not exactly the same as plotting versus timestamps.  Because the current Raspberry Pi is not using a hard real-time implementation for control of sampling intervals, there can be some variability there.   Using sample indices for the X-axis is a simplification that is necessary in order for us to use circular input buffers, which in turn is necessary to keep the plotting functions fast enough to keep up with the input data streams.

### Single Variable Plot Implementation

Note that calling the push() method puts newly received data into the circular buffers.  It does NOT update plot graphics.  That is done with the update() method.  Those two are really the only methods that need to be called from outside the class itself.

In [19]:
class x_plot():
    def plotCallback(self, change):
        newState = change['new']
        if newState:
            self.idxrange = change.new
            self.minIndex=min(self.idxrange)
            self.maxIndex=max(self.idxrange)
            self.db_brush.value = '%d to %d' % (self.minIndex, self.maxIndex)
    
    def __init__(self, maxLen, title, xAxisLabel, yAxisLabel, logger, loggingController):
        self.title = title
        self.logger = logger
        self.loggingController = loggingController
        self.db_brush = widgets.HTML(value='[]')
        self.lock = threading.Lock()
        self.ts = bq.LinearScale()
        self.ys = bq.LinearScale()
        self.maxLen = maxLen
        self.xData  = plot_buffer(maxLen)
        self.tData = np.linspace(-maxLen,0,maxLen)       
        self.xLine = bq.Lines(x=self.tData, y=[self.xData.b], scales={'x':self.ts, 'y':self.ys}, labels=['X'], colors=['red'], display_legend=True)        
        lw = 0.7
        self.xLine.stroke_width=lw      
        def_tt = bq.Tooltip(fields=['x', 'y'], formats=['2.3f', '2.2f'], labels=['Time', 'X'])
        
        self.ax_x = bq.Axis(scale=self.ts, label=xAxisLabel)
        self.ax_y = bq.Axis(scale=self.ys, orientation='vertical', label=yAxisLabel)
        self.figure = bq.Figure(marks=[self.xLine], \
                                axes=[self.ax_x, self.ax_y], title=title)
        
        panzoomx = bq.PanZoom(scales={'x': [self.ts]})
        panzoomy = bq.PanZoom(scales={'y': [self.ys]})
        intervalSelector = bq.interacts.BrushIntervalSelector(scale=self.ts, marks=[self.xLine])
        self.xLine.observe(self.plotCallback, names=['selected'])
        
        from collections import OrderedDict
        selection_interacts = widgets.ToggleButtons(options=OrderedDict([('PanZoomX', panzoomx), ('PanZoomY', panzoomy),
                    ('BrushIntervalSelector', intervalSelector), ('None', None)]))
        link((selection_interacts, 'value'), (self.figure, 'interaction'))
        self.figure.layout.height = '400px'
        self.figure.layout.width = '70%'

        plotButton = widgets.Button(
            description='Download Plot',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Will dump current plot into a file on your computer',
            icon=''
        )
        plotButton.on_click(self.on_plotButton_clicked)
        
        csvDumpButton = widgets.Button(
            description='Dump selected to .csv',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Will dump plot points selected with the Brush Interval Selector to a .csv file',
            icon=''
        )
        csvDumpButton.on_click(self.on_csvDumpButton_clicked)
        
        self.widget = widgets.HBox([self.figure, widgets.VBox( [ \
                widgets.Label("Plot Interactions"), selection_interacts, self.db_brush, \
                widgets.Label("Other Functions"), plotButton, csvDumpButton \
                ])])
        
    def on_plotButton_clicked(self, b):
        fn = self.title+'.png'
        self.figure.save_png(fn)
        
    def on_csvDumpButton_clicked(self, b):
        fn = self.loggingController.file_dialog.selected
        if self.minIndex >= self.maxIndex:
            self.logger.info('You must select a range using the Brush Interval Selector first')
        else:
            r = self.maxIndex - self.minIndex
            t = np.linspace(0,r,r) #/self.sampleRate  
            x = self.xLine.y[self.minIndex:self.maxIndex]
            d = np.vstack((t, x)).T
            headerLine = 'SampleNum, ' + self.title
            np.savetxt(fn, d, delimiter=',', header=headerLine, fmt='%4.3e')
            self.logger.info('Saved: %s' % fn)
    
    def set_visible(self, x):
        self.xLine.visible = x
        self.xLine.display_legend = x

    def push(self,x):
        with self.lock:
            self.xData.push(x)

    def update(self):
        with self.lock:
            x = self.xData.unwrapped()
            self.xLine.y = []
            self.xLine.y = x

numPlotPoints = 256
tempPlot = x_plot(numPlotPoints, 'Temperature', 'Time (sample intervals)', 'Temperature (C)', logger, loggingController)
rHPlot = x_plot(numPlotPoints, 'rH', 'Time (sample intervals)', 'rH (%)', logger, loggingController)
pressurePlot = x_plot(numPlotPoints, 'Pressure', 'Time (sample intervals)', 'inches Mercury', logger, loggingController)

### Tri-Axial Plot Implementation

Again, the tri-axial plot implementation is essentially just a superset of the sing variable implementation.  Note the addition of extra visibility controls for X, Y, Z and vector magnitude.

In [20]:
class xyz_plot():
    def plotCallback(self, change):
        newState = change['new']
        if newState:
            self.idxrange = change.new
            self.minIndex=min(self.idxrange)
            self.maxIndex=max(self.idxrange)
            self.db_brush.value = '%d to %d' % (self.minIndex, self.maxIndex)
    
    def __init__(self, maxLen, title, xAxisLabel, yAxisLabel, logger, loggingController):
        self.title = title
        self.logger = logger
        self.loggingController = loggingController
        self.db_brush = widgets.HTML(value='[]')
        self.lock = threading.Lock()
        self.ts = bq.LinearScale()
        self.xs = bq.LinearScale()
        self.ys = bq.LinearScale()
        self.zs = bq.LinearScale()
        self.vms = bq.LinearScale()
        self.maxLen = maxLen
        self.xData  = plot_buffer(maxLen)
        self.yData  = plot_buffer(maxLen)
        self.zData  = plot_buffer(maxLen)
        self.vmData = plot_buffer(maxLen)
        self.tData = np.linspace(-maxLen,0,maxLen)       
        self.xLine = bq.Lines(x=self.tData, y=[self.xData.b], scales={'x':self.ts, 'y':self.ys}, labels=['X'], colors=['red'], display_legend=True)        
        self.yLine = bq.Lines(x=self.tData, y=[self.yData.b], scales={'x':self.ts, 'y':self.ys}, labels=['Y'], colors=['blue'], display_legend=True)        
        self.zLine = bq.Lines(x=self.tData, y=[self.zData.b], scales={'x':self.ts, 'y':self.ys}, labels=['Z'], colors=['black'], display_legend=True)
        lw = 0.7
        self.xLine.stroke_width=lw      
        self.yLine.stroke_width=lw
        self.zLine.stroke_width=lw  
        def_tt = bq.Tooltip(fields=['x', 'y'], formats=['2.3f', '2.2f'], labels=['Time', 'VM'])
        self.vmLine = bq.Lines(x=self.tData, y=[self.vmData.b],  tooltip=def_tt, unhovered_style={'opacity': 0.5}, \
            scales={'x':self.ts, 'y':self.ys}, labels=['VM'], display_legend=True)
        self.vmLine.colors=['DarkOrange']
        self.vmLine.fill = 'bottom'
        self.vmLine.fill_opacities = [0.10]
        self.vmLine.stroke_width=1
        self.vmLine.marker = 'circle'
        
        self.ax_x = bq.Axis(scale=self.ts, label=xAxisLabel)
        self.ax_y = bq.Axis(scale=self.ys, orientation='vertical', label=yAxisLabel)
        self.figure = bq.Figure(marks=[self.xLine, self.yLine, self.zLine, self.vmLine], \
                                axes=[self.ax_x, self.ax_y], title=title)
        
        panzoomx = bq.PanZoom(scales={'x': [self.ts]})
        panzoomy = bq.PanZoom(scales={'y': [self.ys]})
        intervalSelector = bq.interacts.BrushIntervalSelector(scale=self.ts, marks=[self.xLine, self.yLine, self.zLine, self.vmLine])
        #intervalSelector.observe(self.plotCallback, names=['selected'])
        self.xLine.observe(self.plotCallback, names=['selected'])
        
        from collections import OrderedDict
        selection_interacts = widgets.ToggleButtons(options=OrderedDict([('PanZoomX', panzoomx), ('PanZoomY', panzoomy),
                    ('BrushIntervalSelector', intervalSelector), ('VM Tooltips', None)]))
        link((selection_interacts, 'value'), (self.figure, 'interaction'))
        self.figure.layout.height = '400px'
        self.figure.layout.width = '70%'
        self.visibility_controls = interactive(self.set_visible, x=True, y=True, z=True, vm=False)

        plotButton = widgets.Button(
            description='Download Plot',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Will dump current plot into a file on your computer',
            icon=''
        )
        plotButton.on_click(self.on_plotButton_clicked)
        
        csvDumpButton = widgets.Button(
            description='Dump selected to .csv',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Will dump plot points selected with the Brush Interval Selector to a .csv file',
            icon=''
        )
        csvDumpButton.on_click(self.on_csvDumpButton_clicked)
        
        self.widget = widgets.HBox([self.figure, widgets.VBox( \
                [widgets.Label("Visibility Controls"), self.visibility_controls, \
                widgets.Label("Plot Interactions"), selection_interacts, self.db_brush, \
                widgets.Label("Other Functions"), plotButton, csvDumpButton \
                ])])
        
    def on_plotButton_clicked(self, b):
        fn = self.title+'.png'
        self.figure.save_png(fn)
        
    def on_csvDumpButton_clicked(self, b):
        fn = self.loggingController.file_dialog.selected
        if self.minIndex >= self.maxIndex:
            self.logger.info('You must select a range using the Brush Interval Selector first')
        else:
            r = self.maxIndex - self.minIndex
            t = np.linspace(0,r,r) #/self.sampleRate  
            x = self.xLine.y[self.minIndex:self.maxIndex]
            y = self.yLine.y[self.minIndex:self.maxIndex]
            z = self.zLine.y[self.minIndex:self.maxIndex]
            d = np.vstack((t, x, y, z)).T
            np.savetxt(fn, d, delimiter=',', header='SampleNum, X, Y, Z', fmt='%4.3e')
            self.logger.info('Saved: %s' % fn)
    
    def set_visible(self, x,y,z,vm):
        self.xLine.visible = x
        self.xLine.display_legend = x
        self.yLine.visible = y
        self.yLine.display_legend = y
        self.zLine.visible = z
        self.zLine.display_legend = z
        self.vmLine.visible = vm
        self.vmLine.display_legend = vm
    def push(self,x,y,z):
        with self.lock:
            self.xData.push(x)
            self.yData.push(y)
            self.zData.push(z)
            self.vmData.push(np.sqrt(x*x + y*y + z*z))
    def update(self):
        with self.lock:
            x = self.xData.unwrapped()
            self.xLine.y = []
            self.xLine.y = x
            
            y = self.yData.unwrapped()
            self.yLine.y = []
            self.yLine.y = y
            
            z = self.zData.unwrapped()
            self.zLine.y = []
            self.zLine.y = z
            
            vm = self.vmData.unwrapped()
            self.vmLine.y = []
            self.vmLine.y = vm

numPlotPoints = 256
accelPlot = xyz_plot(numPlotPoints, 'Acceleration', 'Time (sample intervals)', 'Acceleration (g\'s)', logger, loggingController)
magPlot = xyz_plot(numPlotPoints, 'Magnetic_Field', 'Time (sample intervals)', 'Magnetic Field (uT\'s)', logger, loggingController)
gyroPlot = xyz_plot(numPlotPoints, 'Angular_Rotation', 'Time (sample intervals)', 'Rotation (dps)', logger, loggingController)

def update_plot():
    global accelPlot, magPlot, gyroPlot, tempPlot, rHPlot, pressurePlot, tab, skip_plot_updates
    if not skip_plot_updates:
        if tab.selected_index==1:
            accelPlot.update()
        elif tab.selected_index==2:
            magPlot.update()
        elif tab.selected_index==3:
            gyroPlot.update()
        elif tab.selected_index==4:
            tempPlot.update()
        elif tab.selected_index==5:
            rHPlot.update()
        elif tab.selected_index==6:
            pressurePlot.update()
    return

## Tabbed Interface

Here's where we pull together the various functions into the tabbed interface.  The "gui" variable represents the top level of the graphical hierarchy.

In [21]:
tab_titles = ['Main', 'Accelerometer', 'Magnetometer', 'Gyroscope', 'Temperature', 'rH', 'Pressure']
tab = widgets.Tab()
for i in range(len(tab_titles)):
    tab.set_title(i, tab_titles[i])
commonPlotControls = widgets.VBox([widgets.HBox([toggle1, toggle2]), loggingController])
tab.children = [main, \
               widgets.VBox([accelPlot.widget, commonPlotControls], layout={'border':'1px solid black'}), \
               widgets.VBox([magPlot.widget, commonPlotControls], layout={'border':'1px solid black'}), \
               widgets.VBox([gyroPlot.widget, commonPlotControls], layout={'border':'1px solid black'}), \
               widgets.VBox([tempPlot.widget, commonPlotControls], layout={'border':'1px solid black'}), \
               widgets.VBox([rHPlot.widget, commonPlotControls], layout={'border':'1px solid black'}), \
               widgets.VBox([pressurePlot.widget, commonPlotControls], layout={'border':'1px solid black'}) \
               ]
layout = {
    'width': '100%',
    'border': '1px solid black',
}
gui = widgets.VBox([tab, handler.out], layout=layout)

# Socket and GUI interface code
Here we have a bit of code to pull together a few of the top level objects.

## Process sensor data packet
The read_packet() function is called by the SocketClient whenever a new sensor data packet is received.  It transfers data from the ServerClient to the various GUI displays and also updates the packet counter. 

In [22]:
last_packet = None
def read_packet(packet):
    global logger
    try:
        global packet_data
        last_packet = packet
        update_sensors(packet)
        increment_counter()  # update our packet counter
    except ex:
        #print('Exception in read_packet')
        logger.error("Exception in function read_packet1: {0}".format(str(ex)))

## update_sensors() & update_sensors_display()
update_sensors() pushes new data points into the plot objects and also records a new line in the logger output file (if logging is enabled).

update_sensors_display() is called periodically to actually update the GUI visual outputs.

In [23]:
def update_sensors(packet):
    global last_packet, logger, loggingController
    last_packet=packet
    try:
        packetTime = packet['time']
        accX = packet['accelX']
        accY = packet['accelY']
        accZ = packet['accelZ']
        magX = packet['magX']
        magY = packet['magY']
        magZ = packet['magZ']
        gyroX = packet['gyroX']
        gyroY = packet['gyroY']
        gyroZ = packet['gyroZ']
        temp = packet['temp']
        pressure = packet['pressure']
        rH = packet['rH']
        accelPlot.push(accX, accY, accZ)
        magPlot.push(magX, magY, magZ)
        gyroPlot.push(gyroX, gyroY, gyroZ)
        tempPlot.push(temp)
        rHPlot.push(rH)
        pressurePlot.push(pressure)
        loggingController.log(packetTime, accX, accY, accZ, magX, magY, magZ, gyroX, gyroY, gyroZ, temp, pressure, rH)
    except ex:
        logger.error("Exception in function update_sensors: {0}".format(str(ex))) 
        
def update_sensors_display():
    global last_packet
    packet = last_packet
    if not packet == None:
        accX = packet['accelX']
        accY = packet['accelY']
        accZ = packet['accelZ']
        magX = packet['magX']
        magY = packet['magY']
        magZ = packet['magZ']
        gyroX = packet['gyroX']
        gyroY = packet['gyroY']
        gyroZ = packet['gyroZ']
        temp = packet['temp']
        pressure = packet['pressure']
        rH = packet['rH']
        update_accel_row(accX, accY, accZ)
        update_mag_row(magX, magY, magZ)
        update_gyro_row(gyroX, gyroY, gyroZ)
        #print('pressure={0}'.format(pressure))
        #print('rh={0}'.format(rH))
        #print('temp={0}'.format(temp))
        update_sensor_trhp(temp, rH, pressure)

gui_updaters.append(update_sensors_display)
gui_updaters.append(update_plot)


# Display the Raspberry Pi Data Collection GUI
And finally, let's display the final application.  Please note that it may take a number of seconds after pressing the "Connect to Raspberry Pi" button before the connection is established and acknowledged.  Since that is a toggle button, it pays to be patient.  Note that the RPi LED display will cease displaying its IP address and will instead alternate between all green and white once data is being streamed.  Clicking the button a second time will terminate the connection.

In [24]:
display(gui)